In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [2]:
# !pip install --upgrade torchtext
# !pip install --upgrade torch

from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch import optim
import torch.nn as nn 
import torch.nn.functional as F

import torchtext
import numpy as np
import pandas as pd
from random import randrange, choice
import random
from tqdm import tqdm




In [43]:
torch.cuda.is_available()


False

In [4]:
# from google.colab import drive
# drive.mount("/content/gdrive")


pd.set_option("display.max_colwidth", -1)
# path = "/content/gdrive/My Drive/data/amazon/amazon-reviews-max50-37k.csv"
path = "data/amazon-reviews-max50-37k.csv"
df = pd.read_csv(path)

<ipython-input-4-d057213c6eb1>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [5]:
df.head()

,source,target
0,"Since my first trip to Jamaica, I fell in love with the coffee. The delivery date was right on and we had Blue Mountain coffee on Christmas morning! For a treat, try drinking it with sweetened condensed milk!",Jamaican all the way!
1,"this lavish oatmeal in dark chocolate tastes great. the oatmeal is very convenient and quick to make. it tastes great, like cocoa, not exactly like a rich dark chocolate, but still yummy.",yum
2,"I was truly expecting better from Mrs. Dash, but this is nothing more than vinegary glycerin with some unrecognized flavoring. There is absolutely NO oriental flavor whatsoever. Do yourself a favor and go buy some Balsamic vinegar, it tastes the same.",Don't waste your time
3,"These are my favorite cookies, I have one almost every day. It's hard to believe that they are vegan, they taste much better than most non-vegan cookies!",I love these cookies!
4,"Two of the most yummy, sinful flavors in one healthy package, who knew? Love it for the late afternoon pick me up when in fade mode! Thank you ZICO!",decadence made healthful


In [6]:
df.describe()

,source,target
count,37089,37089
unique,32851,26076
top,"It was touted by a friend as a sleep inducer, so I tried it and it works. It tastes kinda mediciney, but it definitely does the job.",Delicious!
freq,8,207


In [7]:
len(df)

37089

In [8]:
from torchtext.data import Field, TabularDataset
import spacy

TEXT = Field(use_vocab=True, init_token="<SOS>", eos_token="<EOS>", 
               fix_length=25, tokenize="spacy",
               include_lengths=True, batch_first=True, lower=True,
               is_target=False
               )

fields = [("source", TEXT),
          ("target", TEXT)]

data = TabularDataset(path, "CSV", fields, skip_header=True)
                     

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the mos

In [9]:
train_data, test_data = data.split(.70)
train_data, val_data = train_data.split(.90)

In [10]:
len(train_data), len(val_data), len(test_data)

(23366, 2596, 11127)

Build vocab

In [11]:
# TEXT.build_vocab(train_data, vectors="fasttext.en.300d") 
# TEXT.build_vocab(train_data, vectors="fasttext.en.300d") 
# TEXT.build_vocab(train_data, vectors="glove.6B.100d", max_size=10000, min_freq=3)
TEXT.build_vocab(train_data, max_size=10000, min_freq=3)

#  Next time max_size=10000, min_freq=3 ?


In [12]:
print(f"Unique tokens in source/target vocabulary: {len(TEXT.vocab)}")

Unique tokens in source/target vocabulary: 8464


In [13]:
test_data[0].__dict__.keys()

dict_keys(['source', 'target'])

In [14]:
train_data[0].source, train_data[0].target

(['i',
  'have',
  'yet',
  'to',
  'find',
  'a',
  'kashi',
  'product',
  'that',
  'i',
  'do',
  "n't",
  'like',
  ';',
  'and',
  'this',
  'cereal',
  'is',
  'especially',
  'tasty',
  '.',
  ' ',
  'i',
  'had',
  'already',
  'tried',
  'it',
  'before',
  'ordering',
  'from',
  'amazon',
  ',',
  'so',
  'i',
  'knew',
  'it',
  'was',
  '"',
  'safe',
  '"',
  'to',
  'order',
  'six',
  'boxes',
  'at',
  'a',
  'time',
  '!',
  ' ',
  'yummy',
  '.'],
 ['great', 'cereal', '!'])

In [15]:
TEXT.vocab.freqs["the"], TEXT.vocab.freqs["a"], TEXT.vocab.freqs["shit"]

(29761, 20164, 1)

In [16]:
TEXT.vocab.stoi["the"], TEXT.vocab.itos[5]

(5, 'the')

In [17]:
import pprint

batch = ["Eric needs to leave".split()]
pp = pprint.PrettyPrinter(indent=4)
pad = TEXT.pad(batch)
%pprint
print(pad)

Pretty printing has been turned OFF
([['<SOS>', 'Eric', 'needs', 'to', 'leave', '<EOS>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']], [6])


In [18]:
import random

i = random.randrange(0, len(train_data))
print(i)
source = train_data[i].source
TEXT.pad([source])


20938


([['<SOS>', 'i', "'ve", 'been', 'doing', 'bonsai', 'for', 'many', 'years', '...', 'even', 'had', 'a', 'business', 'for', 'a', 'few', 'years.<br', '/>this', 'is', 'a', 'nice', 'tree', '.', '<EOS>']], [25])

In [19]:
train_data[0].source

['i', 'have', 'yet', 'to', 'find', 'a', 'kashi', 'product', 'that', 'i', 'do', "n't", 'like', ';', 'and', 'this', 'cereal', 'is', 'especially', 'tasty', '.', ' ', 'i', 'had', 'already', 'tried', 'it', 'before', 'ordering', 'from', 'amazon', ',', 'so', 'i', 'knew', 'it', 'was', '"', 'safe', '"', 'to', 'order', 'six', 'boxes', 'at', 'a', 'time', '!', ' ', 'yummy', '.']

In [20]:
train_data[0].target

['great', 'cereal', '!']

In [21]:
import random

i = random.randrange(0, len(train_data))
print(i)
source = train_data[i].source
TEXT.process([source])


16117


(tensor([[   2,   41,    9, 2097,   38,    0, 6650,    8,   13,   15,   18,   87,
          154,   14,   71,  109,  427, 1147,   17,  346,    4,    6,   31,   10,
            3]]), tensor([25]))

Define iterator/batcher/loader here note that this is 

Model


In [22]:
class Encoder(nn.Module):
  def __init__(self, config):
    super(Encoder, self).__init__()
    self.source_dim = config["source_dim"] # vocab size/number of classes
    self.embedding_dim = config["embedding_dim"]
    self.hidden_dim = config["hidden_dim"]
    self.layers = config.get("layers", 1)
    self.bidirectional_encoder = config.get("bidirectional_encoder", False)

    # Layers
    self.embedding = nn.Embedding(self.source_dim, self.embedding_dim)
    self.dropout = nn.Dropout(config.get("dropout", 0.0))
    self.gru = nn.GRU(self.embedding_dim, self.hidden_dim, dropout=config.get("dropout", 0.0), num_layers=self.layers, bias=True, batch_first=True, bidirectional=self.bidirectional_encoder)
    # self.gru = nn.GRU(64, 64, dropout=0, num_layers=1, bias=True, batch_first=True, bidirectional=False).to(device)

  def forward(self, source, source_lens=None, pack_padded=False):
    
    """
    :param source: [B, T]
    :param source_lengths: [T]
    :param pack_padded: bool
    :return: output: [B, T, H], hidden: Union([1, B, H] or [2, B, H])
    """
    # print("encoder forward")
    # print(input)

    # Embed source index sequences [B, T] > [B, T, E]
    embedded = self.embedding(source)
    
    # print(embedded)
    
    # Apply dropout
    embedded = self.dropout(embedded)
  
    # Pack if padded sequences and lengths given

    if pack_padded:
        
            
      embedded = pack_padded_sequence(embedded, source_lens, batch_first=True, enforce_sorted=False)

    # Map embedded source [B, T, E] to Union([B, T, H] or [B, T, 2*H], Union([1, B, H] or [2, B, H])
    output, hidden = self.gru(embedded) # TODO: add state tuple

    if pack_padded:
      output, output_lens = pad_packed_sequence(output, batch_first=True)

    # Sum outputs of bidirectional RNN: 
    # [B, T, 2*H] > [B, T, H]
    if self.bidirectional_encoder:
      output = output[:, :, :self.hidden_dim] + output[:, :, self.hidden_dim:] 

    return output, hidden 




In [23]:
class Decoder(nn.Module):
  def __init__(self, config):
    super(Decoder, self).__init__()
    self.source_dim = config["source_dim"] # vocab size/number of classes
    self.embedding_dim = config["embedding_dim"] # Can be different from hidden!?
    self.bidirectional_encoder = config.get("bidirectional_encoder", False)
    self.hidden_dim = config["hidden_dim"]
    # self.hidden_dim = 2 * config["hidden_dim"] if self.bidirectional_encoder else config["hidden_dim"]

    # Layers
    self.embedding = nn.Embedding(self.source_dim, self.embedding_dim)
    self.dropout = nn.Dropout(config.get("dropout", 0.0))
    self.gru = nn.GRU(self.embedding_dim+self.hidden_dim, self.hidden_dim, dropout=config.get("dropout", 0.0), bias=True, batch_first=True)
    self.fc = nn.Linear(self.hidden_dim, self.source_dim) # input classes = output classes
    self.softmax = nn.Softmax(dim=1)
    
    # For attention
    self.source_layer = nn.Linear(self.hidden_dim, self.hidden_dim)
    self.state_layer = nn.Linear(self.hidden_dim, self.hidden_dim)
    self.weight_layer = nn.Linear(self.hidden_dim, 1)

  def forward(self, input, hidden, enc_outputs):
    """
    Process batch inputs at one time step and decode prediction.

    :param input: [B]
    :param hidden: [1, B, H]
    :param context_vec: [1, B, H]
    :return: prediction (B, H), hidden (1, B, H)
    """


    # print("decoder forward")
    # print(input)
    
    # Process the batch inputs only at this timestep t 
    # Input of shape [B], but we want [B, 1] 
    input = input.unsqueeze(1)

    # print(f"input of shape {input.size()}")
    # print(f"context vec of shape {context_vec.size()}")


    # Embed input [B, 1] > [B, 1, E] 
    # Note embedding dimension needs to equal hidden if concatenated
    # But could map to same space
    input = self.embedding(input)
    # print(input)

    # Apply attention and get next input
    # [B, 1, 2*H]
    next_input = self.attention(input, hidden, enc_outputs)

    
    # Sum context vec and input ???padded = field.pad(minibatch)
    # embedded = embedded + context_vec 

    # Apply dropout
    # embedded = self.dropout(embedded)



    # embedded = pack_padded_sequence(embedded, input_lens, batch_first=True, enforce_sorted=False)

    # Feed to RNN
    output, hidden = self.gru(next_input, hidden)
    
    # print(f"output==hidden {output[0][0]==hidden[0][0]}")

    # output, output_lens = pad_packed_sequence(output, batch_first=True)

    # Shape of output: (B, 1, H), we want (B, H)
    output = output.squeeze(1)

    # Get distribution of classes
    output = self.fc(output)

    return output, hidden

  def attention(self, input, hidden, enc_outputs):


    # Calculate attention

    # Calculate attention distribution alpha for all encoder outputs


    # # Find attention using dot product score (source X hidden_target)
    # Didn't work too well
    # # Reshape target hidden: [1, B, H] > [B, H, 1]
    # h_t = hidden.view(-1, self.hidden_dim, 1)
    # # print(f" s_t: {s_t.size()}")

    # # For batch, perform matrix mult of outputs hidden states and hidden state (s_t) at each time step
    # # [B, T, H] X [B, H, 1] = [B, T, 1]
    # # print(f" outputs: {outputs.size()}")
    # score = enc_outputs.bmm(h_t)
    
    # # Reshape [B, T, 1] > [B, 1, T] and take softmax   
    # alphas = self.softmax(score.transpose(1,2))
    # # print(f" alphas: {alphas.size()}")

    # # Find attention. Take weighted sum of hidden states (i.e. linear combo)
    # # [B, 1, T] X [B, T, H] = [B, 1, H]
    # context_vec = alphas.bmm(enc_outputs)

    # Concatenate next input and context
    # [B, 1, H] ; [B, 1, H] = [B, 1, 2*H]
    # print(f" context: {context_vec.size()}")
    # print(f" in: {input.size()}")



    # Find attention using Bahdenah
    # tanh(Vh_s + Wh_t + b) 

    # Project source layer: Vh_s
    # [B, T, H] > [B, T, H]
    source_layer = self.source_layer(enc_outputs) 
    
    # Reshape target hidden: [1, B, H] > [B, 1, H]
    h_t = hidden.view(-1, 1, self.hidden_dim)

    # Project target hidden state to layer
    # [B, 1, H] > [B, 1, H]
    state_layer = self.state_layer(h_t)
    
    # tanh(Vh_s + Wh_t + b) 
    # [B, T, H] > [B, T, H]
    score = torch.tanh(source_layer + state_layer)

    # Project to weight layer with 1 unit
    # [B, T, H] > [B, T, 1]
    weights = self.weight_layer(score)

    # print(weights[0])

    weights = self.softmax(weights)

    # print(weights[0])
    # print(weights.size())
    # print(torch.sum(weights, dim=1)[0])

    # Find attention. Take weighted sum of hidden states (i.e. linear combo)
    # [B, 1, T] X [B, T, H] = [B, 1, H]
    context_vec = weights.view(weights.size(0), 1, -1).bmm(enc_outputs)

    # print(context_vec.size())
    next_input = torch.cat([input, context_vec], dim=-1)


    return next_input
    # Sum hidden and context_vec
    # Didn't seem to work to well
    # hidden = hidden + context_vec.view(1, bs, -1)

    # Reshape [B, 1, H] > [1, B, H]
    # hidden = context_vec.view(1, bs, -1) 

In [24]:
class Seq2Seq(nn.Module):

  def __init__(self, encoder, decoder):
    super(Seq2Seq, self).__init__()
    
    self.bidirectional_encoder = config.get("bidirectional_encoder", False)
    self.softmax = nn.Softmax(dim=-1)

    # Block
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, source, target=None, source_lens=None, pack_padded=False, teacher_force_ratio=.5, predict=False):
    """
    :param source: [B, T]
    :param target: [B, T] 
    :param source_lengths: [T]
    :param pack_padded: bool
    :return: output (B, V), context (B, H), prev_hidden (B, H), weights (B, T)
    """
    
    target_vocab_size = self.encoder.source_dim # here same, but may change for MT
    hidden_size = self.decoder.hidden_dim  # Pain in ass 2 * encoder hidden size

    # Encode, only need output for attention
    # out: [B, T, H], for bidirectional torch automatically sums outputs
    enc_outputs, hidden = self.encoder(source, source_lens, pack_padded)

    # Concatenate hidden states for bidirection encoder
    # [2, B, H] > [1, B, 2*H]
    # if self.bidirectional_encoder:
    #   hidden = torch.cat((hidden[0], hidden[1]), dim=-1)
    #   # Add first dim
    #   hidden = hidden.unsqueeze(0)

    if self.bidirectional_encoder:
      hidden = hidden[0] + hidden[1]
      # Add first dim
      hidden = hidden.unsqueeze(0)

    #Iterate through outputs and decode
    bs = source.size(0)
  
    # Train
    if not predict:
          
      # Get start token for batch, e.g. [SOS ...] 
      target_length = target.size(1) # includes padding (i.e max length) -- to be ignored by loss func
      
      # [B, 1]
      # print(f" target: {target.size()}")
      input = target[:, 0] # SOS
      # print(f" input: {input.size()}")

      # Get all output predictions step by step and store in matrix of shape [B, T, H]
      predictions = torch.zeros(bs, target_length, target_vocab_size).to(device)
      for t in range(target_length):
        
        # Decode
        output, hidden = self.decoder(input, hidden, enc_outputs)

        # Get predicted output sequences for timestep
        prediction = self.softmax(output)
        
        # print("prediction", prediction)

        # [B, H] as softmax distribution
        predictions[:, t] = prediction
        prediction = prediction.argmax(1)

        # print(f"Predicted sequence {prediction}; Target {target[:, t]}")
        
        # Get next input with or without teacher forcing (correction)
        input = target[:, t] if random.random() < teacher_force_ratio else prediction



        # # Decode
        # prediction, hidden = self.decoder(input, hidden, context_vec)

        # # print("prediction", prediction)
        # predictions[:, t] = prediction
        # prediction = prediction.argmax(1)
        # # print(f"Predicted sequence {prediction}; Target {target[:, t]}")
        # input = target[:, t] if random.random() < teacher_force_ratio else prediction

    # Predict
    else:
      input = torch.ones((1,1)) # SOS
      prediction = torch.zeros((1,1))
      predictions = []
      while prediction.item() != 2: # EOS
        prediction, hidden = self.decoder(input, hidden)
        predictions[:, t] = prediction
        prediction = prediction.argmax(1)
        input = prediction
        predictions.append(prediction.item())
        
    return predictions 



In [25]:
def train(model, iterator, criterion, optimizer):

  model.train()
  train_loss = 0

  for batch in train_iter:

    # Zero so gradients don't accumate 
    optimizer.zero_grad()

    # Move tensors to cuda
    source, source_lens = batch.source
    target, target_lens = batch.target
    source = source.to(device)  # [B,T]
    target = target.to(device)  # [B,T]

    # Encode and decode sequence
    predictions = model(source, target, source_lens, pack_padded=True) 
    
    # Create mask
    bs, max_len = target.size() 
    mask = torch.zeros(bs, max_len)
    for i in range(bs):
        mask[i, :target_lens[i]] = 1
    mask = mask.view(-1)
    target = target * masteams
    
    # Truncate first SOS prediction and reshape [B, T, H] > [B*T, H]
    predictions = predictions[:, 1:, :].view(-1, predictions.size(2))
    target = target[:, 1:].view(-1)
    mask = mask[:, 1:].view(-1)

    # print(predictions)
    # print(target)

    loss = criterion(predictions, target)
    
    print(f"Loss before: {loss} and averaged loss over {len(iterator)} samples: {loss/len(iterator)}")
    
    loss_ = 
    
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  return train_loss / len(iterator)


In [26]:
def eval(model, iterator, criterion):

  model.eval()

  with torch.no_grad():

    epoch_loss = 0
    for batch in iterator:
      
      
      # Move tensors to cuda
      source, source_lens = batch.source
      target, target_lens = batch.target
      source = source.to(device)
      target = target.to(device)
      
      predictions = model(source, target, source_lens, pack_padded=True)
      
      # Truncate first SOS prediction
      predictions = predictions[:, 1:, :].reshape(-1, predictions.size(2))
      target = target[:, 1:].reshape(-1)
      # print(f"Predictions of shape {predictions.size()}", predictions)
      # print(f"Target of shape {target.size()}", target)
    
      loss = criterion(predictions, target)

      epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [27]:
import random
    
def random_eval(model, data, r=2):

  model.eval()

  with torch.no_grad():
        
    for _ in range(r):
      
      i = random.randrange(0, len(data))
      print(f"random num: {i}")
      source, target = data[i].source, data[i].target

      print(">", " ".join(source))
      print("=", " ".join(target))

      # Perform preprocessing (indexing, etc.)
      source, source_lens = TEXT.process([source])
      target, target_lens = TEXT.process([target])
      print(source)
      print(target)
      print(source_lens)
      print(target_lens)
        
        
      # Move tensors to cuda
      source = source.to(device)
      target = target.to(device)
      
      predictions = model(source, target, source_lens, pack_padded=True)
      
      # Truncate first SOS prediction
      predictions = predictions[:, 1:, :]
      target = target[:, 1:]

      # Get predicted sequences [B, T]
      predicted_seqs = torch.argmax(predictions, -1)

      for i in range(len(predicted_seqs)):
        # for t in range(len(predicted_seqs[i])):
        print(f"  {' '.join([TEXT.vocab.itos[t] for t in predicted_seqs[i,:-1]])}")

      print(f"Predictions of shape {predictions.size()}", predictions)
    
      print(f"Predicted seq of shape {predicted_seqs.size()}", predicted_seqs)

      print(f"Target of shape {target.size()}", target)
    

## Retrain, set new params

In [28]:
# Instantiate model


config = dict(
    source_dim=len(TEXT.vocab),
    embedding_dim=512,
    hidden_dim=512,
    layers=1,
    dropout=.5,
    bidirectional_encoder=True,
    bs=32,
    epochs=25,
    lr=.001
)
print(f"Config: {config}")



Config: {'source_dim': 8464, 'embedding_dim': 32, 'hidden_dim': 32, 'layers': 1, 'dropout': 0.5, 'bidirectional_encoder': True, 'bs': 32, 'epochs': 1, 'lr': 0.001}


In [29]:
from torchtext.data import BucketIterator


train_iter, val_iter, test_iter = BucketIterator.splits((train_data, val_data, test_data),
                                                        batch_sizes=(config.get("bs"),config.get("bs"),config.get("bs")),
                                                        sort_within_batch=True,
                                                        sort_key=lambda x: len(x.source),
                                                        device="cpu")

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [30]:
batch = next(iter(train_iter))
batch.source

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


(tensor([[   2,   18,  859,  153,   13,   74,    6,   29,    9, 1048,    7,   10,
           29,  166,   69,    4,   12,  199,   15,   11,   11,   12,   19,   16,
            3],
        [   2,   13, 3530,   15,  409,  183,  169,    4,   10,  110,   37,   71,
          137,   82,    4,  239,   20,  342,    8,  259,   20,  743,   48,  309,
            3],
        [   2,  769,    5,  235,  219,   30,   51,    5, 5244,  702,    7,   27,
           81,   51, 3389,    4,   12,    6,   72,   14,  962,   39,   92,    4,
            3],
        [   2,   25,  155,   24,    9,  810, 5894,  737,   75,    7,   18,  279,
            8,   27,   63,   31,   39,    4,   12,    5,   40, 1550,  336,    8,
            3],
        [   2,    6,   52,   37,  225,   68,  172,    5,  486,    8,   23,   13,
          116,  514,   15, 2612,    4,   12,    5,   90,  391,   58, 1736,   30,
            3],
        [   2, 5835, 2598,   15,   22,  334,  456,    6,  610,    4,   12,   13,
           29,    9,   21,  

In [31]:

list(train_iter)[-1].source

(tensor([[   2,    6,  307, 1132,   58, 2496,    8,   25,  164,   64,   54,  821,
            7,   21,    4,   27,   81,   34,  383,    7, 3170,    4,    3,    1,
            1],
        [   2,    5,  242,   79,   83,   25,  155,    4,   23,   27,   24,  393,
         2749,    7,  175,   16,  242,  131,   14,  111,    4,    3,    1,    1,
            1],
        [   2,  166,  333,   19,    7,   21,  453,   14,   80,   67,  245, 1779,
           36,   57,   52,  101,  482,   33,   57,    0,    4,    3,    1,    1,
            1],
        [   2,   25,   24,   18,   87,  710,  405,  949,  368,  185,   44,   21,
          125,   36,   22,   50,   47,   35,   36,  201,   11,    3,    1,    1,
            1],
        [   2, 3259,   29,  376,    7,    6,  165,  235,  341,  528,  169,    4,
           12,    5, 1173, 1515,   15,   69,   11,   11,    3,    1,    1,    1,
            1],
        [   2,    6,   86,   13,   30,   18, 5401,  570,  602,    4,  564,   38,
          393,  260,    7,  

In [32]:

TEXT.vocab.itos[25]

'these'

In [33]:
list(train_iter)[-1].target  # Batch > sents, lengths

(tensor([[   2,  157,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,   69,   11,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,   46,  489,  112,   11,   11,   11,    3,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,  114,    7,  116,   11,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,   68,   28,   32, 2928, 3332,    8,   13,   53, 1088,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,   63,  119,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,  

In [34]:
encoder = Encoder(config).to(device)
decoder = Decoder(config).to(device)
model = Seq2Seq(encoder, decoder).to(device) 
# model = torch.load(path)

for n, p in model.named_parameters():
  if p.requires_grad:
    print(n, p, p.size())

encoder.embedding.weight Parameter containing:
tensor([[ 1.1736, -1.6631, -0.4518,  ...,  0.0690,  1.0551,  0.7782],
        [ 1.8077,  1.0078,  0.5353,  ...,  1.5839, -2.2235, -0.2189],
        [-1.1051, -0.2164,  0.0673,  ..., -0.7295, -0.1960,  1.5699],
        ...,
        [ 0.3608,  0.8543,  0.0910,  ..., -0.3660,  0.9582,  1.1644],
        [ 1.5009,  0.9670,  1.7449,  ..., -1.2589, -0.6963,  0.3529],
        [ 0.9677,  0.8958, -0.3436,  ..., -0.5509, -0.0756, -1.4694]],
       requires_grad=True) torch.Size([8464, 32])
encoder.gru.weight_ih_l0 Parameter containing:
tensor([[-0.0553, -0.0358,  0.0166,  ...,  0.1285,  0.0167,  0.0793],
        [-0.0984,  0.0552,  0.0791,  ..., -0.1457, -0.1589,  0.1010],
        [ 0.0441,  0.1551, -0.0932,  ..., -0.1481,  0.1551, -0.0915],
        ...,
        [ 0.1451,  0.1562, -0.1232,  ...,  0.1406,  0.1049,  0.0093],
        [ 0.0973, -0.0706,  0.0533,  ..., -0.1684,  0.0906, -0.0953],
        [ 0.0487, -0.0044,  0.0564,  ...,  0.1512,  0.1565,

/home/liefe/.virtualenvs/s2s/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


**Training loop**



In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.Adam(model.parameters())

def loss_fn(predicted, target):
    
    

for epoch in tqdm(range(1, (config.get("epochs") + 1))):

  train_loss = train(model, train_iter, criterion, optimizer)
  dev_loss = eval(model, val_iter, criterion)
  print(f"\nTraining loss for epoch {epoch}:\t {train_loss}")
  print(f"\nDev loss for epoch {epoch}:\t {dev_loss}")

  random_eval(model, train_data, 10)
  
# path = "/content/gdrive/My Drive/data/amazon/models/model.pt"
# torch.save(model, path)


100%|██████████| 1/1 [04:20<00:00, 260.74s/it]


Training loss for epoch 1:	 8.862576512077112

Dev loss for epoch 1:	 8.84089662970566
random num: 15372
> is the best tea i have ever had !   you do not need sweetner of any kind and it tastes wonderful , i highly recommend this product !
= 
tensor([[   2,   13,   15,    5,   46,   45,    6,   26,  112,   72,   11,   12,
           28,   52,   22,  229, 1522,   17,  137,  353,    7,   10,   99,  138,
            3]])
tensor([[   2, 6912,  362,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])
tensor([25])
tensor([4])
  <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Predictions of shape torch.Size([1, 24, 8464]) tensor([[[1.2947e-05, 4.0425e-08, 2.6654e-08,  ..., 1.2413e-08,
          3.1879e-08, 1.3819e-08],
         [1.2886e-05, 4.0418e-08, 2.6563e-08,  ..., 1.2346e-08,
          3.1715e-08, 1

In [36]:
len(TEXT.vocab)

8464

In [22]:
source = torch.randint(10, (4,10))
target  = torch.randint(10, (4,10)) 


In [23]:
source

tensor([[9, 5, 1, 6, 6, 8, 3, 1, 9, 4],
        [8, 7, 5, 6, 3, 1, 3, 0, 0, 9],
        [2, 5, 3, 6, 6, 1, 5, 2, 4, 9],
        [2, 7, 7, 4, 6, 7, 5, 0, 6, 0]])

In [24]:
target

tensor([[1, 6, 1, 7, 4, 6, 9, 2, 8, 0],
        [8, 3, 9, 5, 9, 8, 8, 5, 6, 4],
        [1, 2, 4, 3, 3, 8, 2, 6, 6, 4],
        [1, 1, 1, 2, 8, 4, 0, 6, 6, 4]])

In [25]:
target_lens = [4, 6, 3, 10]

In [20]:
 target[:,4:] = 1
 target


tensor([[2, 7, 6, 6, 1, 1, 1, 1, 1, 1],
        [2, 5, 2, 3, 1, 1, 1, 1, 1, 1],
        [2, 3, 4, 4, 1, 1, 1, 1, 1, 1],
        [5, 3, 9, 7, 1, 1, 1, 1, 1, 1]])

In [26]:
# Create mask
bs, max_len = target.size() 
mask = torch.zeros(bs, max_len)
for i in range(bs):
    mask[i, :target_lens[i]] = 1
mask

tensor([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [28]:
target

tensor([[1, 6, 1, 7, 4, 6, 9, 2, 8, 0],
        [8, 3, 9, 5, 9, 8, 8, 5, 6, 4],
        [1, 2, 4, 3, 3, 8, 2, 6, 6, 4],
        [1, 1, 1, 2, 8, 4, 0, 6, 6, 4]])

In [27]:
target * mask

tensor([[1., 6., 1., 7., 0., 0., 0., 0., 0., 0.],
        [8., 3., 9., 5., 9., 8., 0., 0., 0., 0.],
        [1., 2., 4., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 2., 8., 4., 0., 6., 6., 4.]])

In [ ]:
    # Truncate first SOS prediction and reshape [B, T, H] > [B*T, H]
    predictions = predictions[:, 1:, :].view(-1, predictions.size(2))
    target = target[:, 1:].view(-1)
    mask = mask[:, 1:].view(-1)

In [34]:
x=torch.rand(4,6
            )
x

tensor([[0.4258, 0.0387, 0.0075, 0.0979, 0.4808, 0.8143],
        [0.5739, 0.2371, 0.4398, 0.4092, 0.3307, 0.1077],
        [0.7814, 0.6574, 0.1097, 0.6844, 0.6188, 0.1062],
        [0.9651, 0.8787, 0.7334, 0.5664, 0.9096, 0.1484]])

In [40]:
ind = torch.randint(x.size(-1), (4,))
ind

tensor([1, 3, 0, 5])

In [45]:
torch.gather(x, 1, ind.unsqueeze(-1))

tensor([[0.0387],
        [0.4092],
        [0.7814],
        [0.1484]])